In [1]:
import numpy as np
import pandas as pd
import transformers
from transformers import AutoModel, BertTokenizerFast, BertConfig, BertModel
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.autograd
import torch.nn.functional as F
import torch
from torch import Tensor
from torch.nn.parameter import Parameter,   UninitializedParameter
from torch.nn import functional as F

In [2]:
def func_collecting_tensors(step, tensor1, tensor2=None):
    '''собирает в тензор     '''
    if step == 0:
        return tensor1.unsqueeze(0)
    else:
        return torch.concatenate((tensor1, tensor2),0)
    


def Tucker_Decomposition(tensor):
    n1, n2, n3 = tensor.shape
    u1, _, _ = torch.svd(torch.reshape(tensor, (n1, -1)))
    u2, _, _ = torch.svd(torch.reshape(torch.permute(tensor, [1, 2, 0]), (n2, -1)))
    u3, _, _ = torch.svd(torch.reshape(torch.permute(tensor, [2, 0, 1]), (n3, -1)))
    return u1, u2, u3


def get_tucker_tensors(dict_layers):
    '''делает словарь где ключом будет слой, а значением будет тензор'''        
    dict_tensor = dict(zip(range(12), [[]]*12))
    for key in dict_layers.keys():
        dict_tensor[key].append(torch.cat(dict_layers[key], 2 ))
    return dict_tensor

In [3]:
class MyLinearFunction(torch.autograd.Function):

        # Note that forward, setseup_context, and backward are @staticmethods
    @staticmethod
    def forward(ctx,input, weight, bias, grads, treshold):
        treshold = treshold
        if ((len(grads) <30 )):

            ctx.save_for_backward(input, weight, bias)
        else:
            u1, U, VT =Tucker_Decomposition(torch.cat(MyLayer.grads))
            ctx.save_for_backward(input,weight, bias, U, VT)
        ctx.size = input.shape[0]

        return  input @ weight.T  + bias
    


    @staticmethod
    def backward(ctx, grad_output):

        if len(ctx.saved_tensors) == 3:
            input,weight, bias = ctx.saved_tensors
            grad_input = grad_weight = grad_bias = None
            if ctx.needs_input_grad[0]:
                grad_input = grad_output @ weight
            if ctx.needs_input_grad[1]:
                grad_weight =  grad_output.T @input
  
            if bias is not None and ctx.needs_input_grad[2]:
                grad_bias = grad_output
            
        elif len(ctx.saved_tensors) == 5:

            input,weight, bias, U, VT = ctx.saved_tensors
            grad_input = grad_weight = grad_bias = None
            if ctx.needs_input_grad[0]:
                grad_input = grad_output @ weight
            if ctx.needs_input_grad[1]:
                grad_weight = grad_output.T @input 
                grad_weight = U @  grad_weight @ VT
                grad_weight = torch.where(torch.abs(grad_weight) >= treshold, grad_weight, 0)
            if bias is not None and ctx.needs_input_grad[2]:
                grad_bias = grad_output
            
        return grad_input, grad_weight, grad_bias, None, None
        
        


In [4]:
class MyLinear(torch.nn.Module):
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.empty((out_features, in_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.grads = []
        self.treshold = 1e-3
        
        
    def from_pretrained(self, nn.Linear):
        
        
        
    def forward(self, x):
            return MyLinearFunction.apply(x, self.weight, self.bias, self.grads, self.treshold)
            

In [5]:
x= torch.ones((1,10))
a = torch.nn.Parameter(torch.randn((10,10)))
b = torch.nn.Parameter(torch.randn((10,10)))
c = torch.nn.Parameter(torch.randn((1, 10)))
u = torch.randn((10,10)) 
vt = torch.randn((10,10))
treshold = 1e-3

In [6]:
MyLayer = MyLinear(10, 10)
MyLayer.weight = nn.Parameter(a, requires_grad = True)
MyLayer.bias = nn.Parameter(c, requires_grad = True)

In [6]:
gt = torch.nn.Parameter(torch.randn((1, 10)))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(MyLayer.parameters())

In [8]:
for i in range(30):
    optimizer.zero_grad()
    output = MyLayer(x)
    loss = criterion(output, gt)
  
    loss.backward()
    MyLayer.grads.append(MyLayer.weight.grad.unsqueeze(0))

    optimizer.step()

In [9]:
optimizer.zero_grad()
output = MyLayer(x)
loss = criterion(output, gt)

loss.backward()

optimizer.step()